In [1]:
import numpy as np
import random

import torch
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path

    params = {
            # dataset
            'dataset': "MNIST",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 5,
            'per_task_examples': np.inf,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 10,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:6' if torch.cuda.is_available() else 'cpu'),
                         
            # sample selection
            'alpha':0.005
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"greedy/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/greedy/dataset=MNIST/seed=1_epoch=5_lr=0.001_alpha=0.005_tau=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = params['per_task_examples'],
                      random_class_idx = False)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = params['per_task_examples'],
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28
    
elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

elif params['dataset'] == 'CIFAR100':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

[0 1 2 3 4 5 6 7 8 9]


In [3]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [4]:
from algorithms import GreedySelection
from metrics import MetricCollector2

backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
algorithm = GreedySelection(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [5]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 99.80985345680122, 'loss': 4.688272798702508e-05, 'std': 0.013935089454286675}
[2] Eval metrics for task 1 >> {'accuracy': 99.89795918367346, 'loss': 2.993500525083384e-05, 'std': 0.10204081632653184}
[3] Eval metrics for task 1 >> {'accuracy': 99.89795918367346, 'loss': 1.9580971925182547e-05, 'std': 0.10204081632653184}
[4] Eval metrics for task 1 >> {'accuracy': 99.89795918367346, 'loss': 1.585140690013731e-05, 'std': 0.10204081632653184}
[5] Eval metrics for task 1 >> {'accuracy': 99.89795918367346, 'loss': 1.4160802735381234e-05, 'std': 0.10204081632653184}
training_task_end
---------------------------- Task 2 -----------------------
len(select_curr_indexes)=2438
For debugging: len(select_curr_indexes)=2438
[6] Eval metrics for task 1 >> {'accuracy': 99.47361323383979, 'loss': 0.00011442569447747359, 'std': 0.0858581317989715}
[6] Eval metrics for task 2 >> {'accuracy': 60.36869

In [6]:
len(benchmark.seq_indices_train[2])

1384

In [7]:
len(benchmark.seq_indices_train[1])

12665

In [8]:
len(algorithm.original_seq_indices_train)

11800

In [9]:
len(set(np.random.choice(12089, 6000, False)))

6000

In [10]:
metric_manager_callback.meters['std'].get_std()

[0.10204081632653184,
 4.755129765088689,
 10.547507988720005,
 12.25344707791861,
 18.392344196899067]

In [11]:
import torch
a = torch.tensor([12, 3])

In [12]:
isinstance(a, torch.Tensor)

True

In [13]:
metric_manager_callback.meters['accuracy'].get_data()

array([[99.898,  0.   ,  0.   ,  0.   ,  0.   ],
       [95.886, 90.658,  0.   ,  0.   ,  0.   ],
       [87.177, 72.4  , 89.817,  0.   ,  0.   ],
       [74.58 , 68.523, 76.254, 93.543,  0.   ],
       [68.616, 53.718, 61.673, 84.148, 72.283]])

In [14]:
metric_manager_callback.meters['accuracy'].compute_overall()

[99.89795918367346,
 93.27201153400446,
 83.13126766743007,
 78.22489382688246,
 68.08752108465289]